In [1]:
from pyspark.sql.types import StructType, StringType, StructField
import pyspark.sql.functions as F
import collections
import functools
import logging
import operator
import tldextract
import json
import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1593586031152_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
NUM_PARTITIONS = 200
INDEX = 'CC-MAIN-2020-16'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def extract_provider(provider_domain):
    try:
        extracted = tldextract.extract(provider_domain)
        if extracted.domain:
            return extracted.domain
    except:
        pass
    return provider_domain


def get_license(_path):
    license = ""
    version = ""

    # how do we handle providers that link to the image for the license? See example below
    # format this case /i.creativecommons.org/l/by-nc-sa/3.0/us/88x31.png
    _path = re.sub(r"^(/i\.creativecommons\.org)", "", _path)

    if _path.startswith("/l/"):
        _path = "/licenses/{}".format(_path.lstrip("/l/"))
    elif _path.startswith("/p/"):
        _path = "/publicdomain/{}".format(_path.lstrip("/p/"))

    pattern = re.compile("/(licenses|publicdomain)/([a-z\-?]+)/(\d\.\d)/?(.*?)")
    if pattern.match(_path.lower()):
        result = re.search(pattern, _path.lower())
        license = result.group(2).lower().strip()
        version = result.group(3).strip()

        if result.group(1) == "publicdomain":
            if license == "zero":
                license = "cc0"
            elif license == "mark":
                license = "pdm"
            else:
                logging.warning("License not detected!")
                return None

        elif license == "":
            logging.warning("License not detected!")
            return None

        return [license, version]

    return None


def get_license_string(license_path):
    license_list = get_license(license_path)
    return "---".join(license_list) if license_list else None


extract_provider_udf = F.udf(extract_provider, StringType())
get_license_string_udf = F.udf(get_license_string, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
provider_df = (
    spark.read.parquet("s3://commonsmapper-v2/data/crawl_index={}".format(INDEX))
    .select(
        "provider_domain",
        "content_path",
        "content_query_string",
        "cc_license",
        "html_metadata",
    )
    .dropDuplicates(
        ['provider_domain', 'content_path', 'content_query_string']
    )
    .drop('content_path')
    .drop('content_query_string')
    .withColumn("images", F.get_json_object("html_metadata", "$.Images"))
    .withColumn("links", F.get_json_object("html_metadata", "$.Links"))
    .drop("html_metadata")
    .repartition(NUM_PARTITIONS)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
base_df = (
    provider_df
    .withColumn("provider_name", extract_provider_udf("provider_domain"))
    .withColumn("cc_license", get_license_string_udf("cc_license"))
)

# Filter for some arbitrary providers to reduce data for testing.
# Comment this out for the real thing.
# base_df = base_df.filter(
#     (base_df.provider_name == 'afpls')
#     | (base_df.provider_name == 'rudn-mr')
#     | (base_df.provider_name == 'attert')
#     | (base_df.provider_name == 'santamariadelparamo')
# ) 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
base_df.cache()
# base_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[provider_domain: string, cc_license: string, images: string, links: string, provider_name: string]

In [7]:
links_all_df = (
    base_df
    .select("provider_name", "links")
    .withColumn("link_map", F.from_json("links", "MAP<STRING,INT>"))
    .select("provider_name", F.explode_outer("link_map"))
    .na.drop()
    .repartition(NUM_PARTITIONS)
    .withColumn("target", extract_provider_udf("key"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
target_df = links_all_df.select(links_all_df.provider_name.alias("target")).distinct()
filtered_targets_links_df = links_all_df.join(target_df, 'target', 'inner').repartition(NUM_PARTITIONS)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
links_json_df = (
    filtered_targets_links_df
    .groupBy("provider_name", "target")
    .agg(F.sum("value").alias("total"))
    .repartition(NUM_PARTITIONS)
    .select("provider_name", F.struct("target", "total").alias("links"))
    .withColumn(
        "total_cc_links", 
        F
        .when(
            F.col("links").target == 'creativecommons',
            F.col("links").total
        )
        .otherwise(0)
    )
    .withColumn(
        "links",
        F
        .when(
            F.col("links").target == 'creativecommons',
            None
        )
        .otherwise(F.col("links"))
    )
    .groupBy("provider_name")
    .agg(
        F.collect_list("links").alias("links"),
        F.sum("total_cc_links").alias("total_cc_links")
    )
    .filter(F.col("total_cc_links") > 0)
    .repartition(NUM_PARTITIONS)
    .withColumn("agg_map_links", F.map_from_entries("links"))
    .select("provider_name", "total_cc_links", F.to_json("agg_map_links").alias("links"))
    .repartition(NUM_PARTITIONS)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
licenses_df = (
    base_df
    .select("provider_name", "cc_license")
    .na.drop()
    .repartition(NUM_PARTITIONS)
    .groupBy("provider_name", "cc_license")
    .agg(F.count("cc_license").alias("total"))
    .repartition(NUM_PARTITIONS)
    .select("provider_name", F.struct("cc_license", "total").alias("cc_license"))
    .groupBy("provider_name")
    .agg(F.collect_list("cc_license").alias("cc_license_arr"))
    .repartition(NUM_PARTITIONS)
    .withColumn("agg_map_licenses", F.map_from_entries("cc_license_arr"))
    .select("provider_name", F.to_json("agg_map_licenses").alias("cc_license"))
    .repartition(NUM_PARTITIONS)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
images_df = (
    base_df
    .select("provider_name", "provider_domain", "images")
    .na.drop()
    .groupBy("provider_name")
    .agg(
        F.first("provider_domain").alias("provider_domain"),
        F.sum("images").alias("images")
    )
    .repartition(NUM_PARTITIONS)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
output_df = (
    links_json_df
    .join(licenses_df, "provider_name", "inner")
    .join(images_df, "provider_name", "inner")
    .repartition(NUM_PARTITIONS)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
output_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[provider_name: string, total_cc_links: bigint, links: string, cc_license: string, provider_domain: string, images: double]

In [12]:
# output_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [22]:
# output_df.show()
output_df.coalesce(1).write.mode("overwrite").options(sep="\t", quote="").csv(
        "s3://commonsmapper/data_viz/{}".format(INDEX)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-22:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 190, in cell_monitor
    update_job_content(job_info_box, job_data)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/updatewidgets.py", line 38, in update_job_content
    update_job_progress_bar(job_progress_bar_holder, job_data)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/updatewidgets.py", line 48, in update_job_progress_bar
    progress_bar.max = new_max
  File "/emr/notebook-env